In [1]:
import os 
import numpy as np
import pandas as pd
import importlib as imp

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

- **p01_c.txt**, the knapsack capacity. <br>
- **p01_w.txt**, the weights of the objects. <br>
- **p01_p.txt**, the profits of each object. <br>
- **p01_s.txt**, the optimal selection of weights. <br>

In [90]:
from utils import tools
tools = imp.reload(tools)

In [3]:
%%time
n = np.arange(1,8)
benchmarks = {k: tools.get_knapsack(n='0'+str(k)) for k in n}

01 is loaded!
02 is loaded!
03 is loaded!
04 is loaded!
05 is loaded!
06 is loaded!
07 is loaded!
Wall time: 29.8 s


In [22]:
benchmarks[1]

{'capacity': [165],
 'weights': [23, 31, 29, 44, 53, 38, 63, 85, 89, 82],
 'profits': [92, 57, 49, 68, 60, 43, 67, 84, 87, 72],
 'optimal': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]}

#TODO
---
1) **BruteForce** Простая эвристика, которая является полиномиальным решением задачи о дробном рюкзаке <br>
2) **Dynamic**          Методом динамического программирования <br>
3) **BoundAndBranches** Методом ветвей и границ <br>
4) **Genetic** Генетическим алгоритмом <br>

In [185]:
from utils import tools
tools = imp.reload(tools)
from algorithm import brute_force
brute_force = imp.reload(brute_force)

In [189]:
i = 7
knap = benchmarks[i]
alg   = brute_force.BruteForce(knap)
opt = alg.solve()
print('BruteForce optimal solution {}'.format(opt))
print('BruteForce optimal profit {}'.format(tools.get_profit(benchmarks[i], opt)))
print('Real optimal profit {}'.format(tools.get_profit(benchmarks[i], benchmarks[i]['optimal'])))

BruteForce optimal solution [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
BruteForce optimal profit 1458
Real optimal profit 1458
